In this notebook we train our model: a BERT-like model with attention flow inspired by BiDAF.

Dependencies:

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

This part should be data loading and processing.

Input: SQuAD dataset handler/url/json

Output: processed dict/list/whatever: train_question, train_context, train_answer

In [2]:
from utils import data_processing
encodings =  data_processing.data_processing("https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json")

Test on the encodings

In [3]:
print(encodings.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


In [10]:
from pytorch_pretrained_bert import BertModel, BertTokenizer
model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ImportError: cannot import name 'BertTokenizerFast' from 'pytorch_pretrained_bert' (C:\Users\Holie\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_pretrained_bert\__init__.py)

In [5]:
input_ids = torch.tensor(encodings['input_ids'], dtype = torch.long)
input_ids.shape

torch.Size([130319, 991])

In [17]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [33]:
embed = nn.Embedding(10,3)

In [34]:
embed(torch.tensor(range(10)))

tensor([[ 0.6905,  0.1964,  1.6562],
        [-0.4224,  0.5683, -0.3049],
        [-0.5149, -0.6370,  0.3069],
        [ 0.0590,  1.4040,  0.8034],
        [ 0.1249,  1.0617,  0.1690],
        [ 1.2919,  0.9869, -1.9232],
        [-1.3960,  0.3178,  1.1482],
        [-0.6398, -0.8063, -1.3722],
        [ 0.3509, -0.8943,  0.7763],
        [-2.6271, -0.1844, -0.5457]], grad_fn=<EmbeddingBackward>)

In [6]:
attention_mask = torch.tensor(encodings['attention_mask'], dtype = torch.long)
print(attention_mask.shape)

torch.Size([130319, 991])


In [7]:
dataset = torch.utils.data.TensorDataset(input_ids, attention_mask)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16)

In [8]:
hidden_states = []
for ids, masks in dataloader:
    count = 0
    ids = ids.to(device)
    masks = masks.to(device)
    hidden_state, _ = model(input_ids=ids, token_type_ids = None, attention_mask = masks)
    hidden_states.append(hidden_state)
    count += 1
    if count == 5:
        break

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

This part should be model construction.

In [ ]:
from layers.bert_plus_bidaf import BERT_plus_BiDAF
""" TODO: implement BERT_plus_BiDAF model class """

This part should be declaration of the optimizer and the loss function. 

In [ ]:
""" TODO: optimizer, and loss function(not urgent)"""

This part should be the definition of training process:

In [ ]:
def train(model, inputs):
""" TODO: determine inputs """

Rest part is for experiments: